In [1]:
import json
import time
import numpy as np
import paho.mqtt.client as mqtt
from fmi_mlc import fmi_gym
from parameters import parameter as PARAMS
from mqtt_publisher import MQTTPublisher

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
PARAMS

{'seed': 1,
 'store_data': True,
 'fmu_step_size': 600.0,
 'fmu_path': 'lab_building_external_control.fmu',
 'fmu_start_time': 0,
 'fmu_warmup_time': 0,
 'fmu_final_time': 2678400,
 'action_names': ['Kitchen_shade_FMU',
  'Bath_shade_1_FMU',
  'Room1_shade_1_FMU',
  'Room1_shade_2_FMU',
  'Bath2_Shade_FMU',
  'Living_shade_1_FMU',
  'Living_shade_2_FMU',
  'Room2_shade1_FMU',
  'Room2_shade_2_FMU'],
 'action_min': array([-1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.,
        -1000.]),
 'action_max': array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.]),
 'observation_names': ['Tin_Kitchen',
  'Tin_Bathroom1',
  'Tin_Room1',
  'Tin_Corridor',
  'Tin_Room',
  'Tin_Linvingroom',
  'T_out',
  'DNI',
  'DistrictHeating',
  'DistrictCooling',
  'Electricity',
  'Tin_Bathroom',
  'Room_shade2',
  'Room_shade1',
  'Bathroom1_shade',
  'Room1_shade1',
  'Room1_shade2',
  'Bathroom_shade1',
  'Living_shade2',
  'Living_shade1'],
 'reward_names': []}

In [3]:
import datetime

YEAR = 2025
def convert_fmu_timestamp(ts, YEAR):
    """
    Convert FMU-relative timestamp (seconds from Jan 1 of a dummy year)
    into a real Unix timestamp using the selected YEAR.
    """
    # Unix timestamp of YEAR-01-01 00:00:00
    base = datetime.datetime(YEAR, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc).timestamp()

    # FMU timestamp is seconds from Jan 1 of the simulation year
    return base + ts


In [4]:
from mqtt_publisher import MQTTPublisher

# Buffer to store the most recent action received from the RL controller
action_buffer = None

def on_connect(client, userdata, flags, rc):
    """
    Callback executed when the MQTT client successfully connects.

    - Prints connection status
    - Subscribes to the topic where the RL controller publishes actions
    """
    print(f"✅ Controlled Sim — Connected (rc={rc})")
    client.subscribe("building/action", qos=1)
    print("✅ Controlled Sim — Subscribed to topic: building/action")

def on_action(client, userdata, msg):
    """
    Callback executed whenever an action is received on the MQTT topic.

    The payload can be:
        - a list (in which case we map it to a dict using PARAMS['action_names'])
        - a dict{name: value} directly
        - anything else (ignored with a warning)
    """
    global action_buffer

    payload = json.loads(msg.payload)

    # Case 1: action sent as a list → convert to dict {action_name: value}
    if isinstance(payload, list):
        action_buffer = {
            name: payload[i]
            for i, name in enumerate(PARAMS['action_names'])
        }

    # Case 2: action already sent as a dict → use directly
    elif isinstance(payload, dict):
        action_buffer = payload

    # Unexpected format → print warning and ignore
    else:
        print("⚠️ Controlled Sim — Unexpected payload:", payload)
        return

# ---------------------------------------------------------------------
# 1) MQTT client that receives actions from the RL controller
# ---------------------------------------------------------------------
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_action
mqtt_client.connect("mosquitto", 1883)
mqtt_client.loop_start()

# ---------------------------------------------------------------------
# 2) Publisher for InfluxDB (via MQTT)
# ---------------------------------------------------------------------
pub = MQTTPublisher(host="mosquitto", port=1883)


✅ Controlled Sim — Connected (rc=0)
✅ Controlled Sim — Subscribed to topic: building/action


/tmp/ipykernel_869/3811467029.py:49: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


In [5]:
# 3) Create and reset the environment
#    This initializes the FMU simulation and provides the first observation.
env = fmi_gym(PARAMS)
obs = env.reset()

# Extract the last row of the internal data (FMU step result)
last = env.data.iloc[-1].to_dict()

# FMU timestamp (seconds)
ts_fmu = last['time']

# Convert to real Unix timestamp
ts = convert_fmu_timestamp(ts_fmu, YEAR)

# 4) Publish initial observation to both the RL controller and Influx RL
#    We only publish the variables listed in PARAMS['observation_names'].
obs_ctrl = {n: last[n] for n in PARAMS['observation_names']}

# Send to RL controller via MQTT
mqtt_client.publish("building/observation", json.dumps(obs_ctrl), qos=1)
print(f"→ Published initial observation: {obs_ctrl}")

# Send the same observation to the Influx RL pipeline
pub.publish_observations_rl(obs_ctrl, ts)
print(f"→ Published initial RL observation at ts={ts}")



[WARNING][Slave] [warning][FMU status:Warning] fmiInitializeSlave: The StopTimeDefined parameter is set to 0. This is not valid. EnergyPlus FMU requires a stop time and will use the stop time 2678400.000000 which is provided.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The sockfd is 61.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The port number is 59015.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: This hostname is a70c621e116a.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: TCPServer Server waiting for clients on port: 59015.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of input variables is 9.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of output variables is 20.

[INFO][Slave] [ok][FMU status:OK] Get input file from resource folder ///tmp/JModelica.org/jm_tmpdbbjm_3s//resources//.

[INFO][Slave] [ok][FMU status:OK] Searching for following pattern .idf

[INFO][Slave] [ok][FMU status:OK] Read directory

Reading input and weather file for preprocessor program.
The IDF version of the input file ///tmp/JModelica.org/jm_tmpdbbjm_3s//resources//lab_building_external_control.idf starts with 9
Successfully finish reading weather file.
This is the Begin Month: 1
Time (0) set is smaller than minimun allowed (1 day). Day will be set to 1.
This is the Day of the Begin Month: 1
This is the End Month: 1
This is the Day of the End Month: 31
This is the New Day of Week: TUESDAY
Running EPMacro...
ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.021
EnergyPlus Starting
EnergyPlus, Version 9.6.0-4b123cf80f, YMD=2025.12.09 09:41
Initializing Response Factors
Calculating CTFs for "GROUNDFLOOR"
Calculating CTFs for "ROOF"
Calculating CTFs for "PARTITION"
Calculating CTFs for "PARTITION_REV"
Calculating CTFs for "INSIDEDOOR"
Calculating CTFs for "EXTERNALWALLBO.287"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Ini

In [6]:
# 5) Main loop (controlled simulation)
done = False
while not done:
    # 5a) Wait until action_buffer is a dict (i.e., an action has arrived)
    while action_buffer is None:
        time.sleep(0.001)

    # 5b) Consume the action from the buffer
    act_dict = action_buffer
    action_buffer = None

    # 5c) Publish actions via publish_actions_rl
    #     ts here is the LAST timestamp used; if you prefer, you can
    #     also compute a fresh timestamp before this publish.
    pub.publish_actions_rl(act_dict, ts)

    # 5d) Step the environment forward
    act = np.array([act_dict[n] for n in PARAMS['action_names']], dtype=np.float32)
    obs, reward, done, _ = env.step(act)

    # Extract latest FMU data row as a dict
    last = env.data.iloc[-1].to_dict()

    # FMU time (seconds from Jan 1 in the FMU time reference)
    ts_fmu = last['time']

    # Convert FMU time to real Unix timestamp using the global YEAR
    ts = convert_fmu_timestamp(ts_fmu, YEAR)

    # 5e) Publish next observation to the controller
    obs_ctrl = {n: last[n] for n in PARAMS['observation_names']}
    mqtt_client.publish("building/observation", json.dumps(obs_ctrl), qos=1)

    # Publish observation, rewards, etc. to RL/Influx using the corrected timestamp
    pub.publish_observations_rl(obs_ctrl, ts)

    # Individual rewards (e.g., each PPD)
    reward_dict = {r: last[r] for r in PARAMS['reward_names']}
    pub.publish_rewards_rl(reward_dict, ts)

# 6) Shutdown
print("✅ Controlled Sim — Episode finished.")
pub.close()
mqtt_client.loop_stop()
mqtt_client.disconnect()


{'Tin_Kitchen': 24.28961574034881, 'Tin_Bathroom1': 23.77939406771481, 'Tin_Room1': 22.62762107763803, 'Tin_Corridor': 25.36773985362892, 'Tin_Room': 26.25531123156834, 'Tin_Linvingroom': 25.74099880134741, 'T_out': 14.36666666666667, 'DNI': 0.0, 'DistrictHeating': 0.0, 'DistrictCooling': 0.0, 'Electricity': 0.0, 'Tin_Bathroom': 24.50097856360936, 'Room_shade2': 0.0, 'Room_shade1': 0.0, 'Bathroom1_shade': 0.0, 'Room1_shade1': 0.0, 'Room1_shade2': 0.0, 'Bathroom_shade1': 0.0, 'Living_shade2': 0.0, 'Living_shade1': 0.0}
{'Tin_Kitchen': 24.22995779158772, 'Tin_Bathroom1': 23.72931378960082, 'Tin_Room1': 22.57441354458533, 'Tin_Corridor': 25.30493690933649, 'Tin_Room': 26.17482843039801, 'Tin_Linvingroom': 25.68859334421348, 'T_out': 14.33333333333333, 'DNI': 0.0, 'DistrictHeating': 51421.30075963086, 'DistrictCooling': 0.0, 'Electricity': 19575.92161265211, 'Tin_Bathroom': 24.44035369691753, 'Room_shade2': 0.0, 'Room_shade1': 0.0, 'Bathroom1_shade': 0.0, 'Room1_shade1': 0.0, 'Room1_shade2

KeyboardInterrupt: 